In [1]:
import geopandas as gpd
import pandas as pd

In [2]:
caltrans_input_gdb_path = r"D:\WORK\wildfire\Interagency-Tracking-System\V2.0\CALTRANS_V2.0\Caltrans_Vegetation_Management_Treatments_2024.gdb"


In [5]:
list(gpd.list_layers(caltrans_input_gdb_path).name)

['Caltrans_Vegetation_Management_RoadsideLandscape_ActivitiesTable_2024',
 'Caltrans_Vegetation_Management_RoadsideLandscape_Treatments_2024',
 'Caltrans_Vegetation_Management_Trees_ActivitiesTable_2024',
 'Caltrans_Vegetation_Management_Tree_Treatments_2024']

In [39]:
validation_gdb_path = r"D:\WORK\wildfire\Interagency-Tracking-System\its\Interagency Tracking System.gdb"
output_gdb_path = r"D:\WORK\wildfire\Interagency-Tracking-System\its\ITSGDB_backup\tmp\Timber_Nonspatial_2020_2025.gdb"
reference_layer_name = 'nfpors_fuels_treatments_polys_enriched2023_20240925'
output_layer_name = 'Timber_Nonspatial_20250221'

In [41]:
gpd.list_layers(validation_gdb_path)

,name,geometry_type
0,WFR_TF_Template,MultiPolygon
1,Broad_Vegetation_Types,MultiPolygon
2,CALFIRE_Ownership_Update,MultiPolygon
3,California,MultiPolygon
4,Counties,MultiPolygon
5,Own_Veg_Region_WUI,MultiPolygon
6,WFRTF_Regions,MultiPolygon
7,WUI,MultiPolygon
8,NFPORSCurrentFYTreatmentsBIA_20240725,Point
9,NFPORSCurrentFYTreatmentsFWS_20240725,Point


In [36]:
output_gdf = gpd.read_file(output_gdb_path, driver='openFileGDB', layer=output_layer_name)
reference_gdf = gpd.read_file(validation_gdb_path, driver='openFileGDB', layer=reference_layer_name)

C:\Users\sky\.conda\envs\its_recode\Lib\site-packages\pyogrio\raw.py:198: RuntimeWarning: driver OpenFileGDB does not support open option DRIVER
  return ogr_read(
C:\Users\sky\.conda\envs\its_recode\Lib\site-packages\pyogrio\raw.py:198: RuntimeWarning: driver OpenFileGDB does not support open option DRIVER
  return ogr_read(


In [37]:
def val_group(reference_gdb_path, 
              output_gdb_path, 
              reference_layer_name,
              output_layer_name,
              col_sum='ACTIVITY_QUANTITY', 
              group=['ADMINISTERING_ORG', 'BROAD_VEGETATION_TYPE', 'PRIMARY_OBJECTIVE', "YEAR_TXT"]):
    
    reference_gdf = gpd.read_file(reference_gdb_path, driver='openFileGDB', layer=reference_layer_name)
    output_gdf = gpd.read_file(output_gdb_path, driver='openFileGDB', layer=output_layer_name)
    
    reference_gdf.columns = [col.upper() for col in reference_gdf.columns]
    
    # mod pfirs reference
    reference_gdf.loc[reference_gdf['ADMINISTERING_ORG'] == 'US Department of Energy', 'ADMINISTERING_ORG'] = 'DOE'
    output_gdf.columns = [col.upper() for col in output_gdf.columns]
    
    out_group = output_gdf[output_gdf['COUNTS_TO_MAS'] == 'YES'][[col_sum] + group].groupby(group).sum().reset_index()
    ref_group = reference_gdf[reference_gdf['COUNTS_TO_MAS'] == 'YES'][[col_sum] + group].groupby(group).sum().reset_index()
    
    merge_group = out_group.merge(ref_group, on=group, how='outer')
    
    print('out of {} matching combinations of {}, there are {} combinations that have different sum activity quantity'.format(len(merge_group),
                                                                                                                              group,
                                                                                                                             len(merge_group[merge_group['ACTIVITY_QUANTITY_x'] != merge_group['ACTIVITY_QUANTITY_y']]),
                                                                                                                             ))
    
    diff_df = merge_group[merge_group['ACTIVITY_QUANTITY_x'] != merge_group['ACTIVITY_QUANTITY_y']]
    diff_df['DIFF'] = diff_df['ACTIVITY_QUANTITY_x'].fillna(0) - diff_df['ACTIVITY_QUANTITY_y'].fillna(0)
    
    return diff_df

In [38]:
diff_df = val_group(validation_gdb_path,
                   output_gdb_path,
                   reference_layer_name,
                   output_layer_name)

out of 21 matching combinations of ['ADMINISTERING_ORG', 'BROAD_VEGETATION_TYPE', 'PRIMARY_OBJECTIVE', 'YEAR_TXT'], there are 0 combinations that have different sum activity quantity


C:\Users\sky\.conda\envs\its_recode\Lib\site-packages\pyogrio\raw.py:198: RuntimeWarning: driver OpenFileGDB does not support open option DRIVER
  return ogr_read(
C:\Users\sky\.conda\envs\its_recode\Lib\site-packages\pyogrio\raw.py:198: RuntimeWarning: driver OpenFileGDB does not support open option DRIVER
  return ogr_read(


In [33]:
diff_df[diff_df['YEAR_TXT'] =='2022']

,ADMINISTERING_ORG,BROAD_VEGETATION_TYPE,PRIMARY_OBJECTIVE,YEAR_TXT,ACTIVITY_QUANTITY_x,ACTIVITY_QUANTITY_y,DIFF
121,NPS,FOREST,OTHER_FUELS_REDUCTION,2022,2566.0,NaN,2566.0
147,NPS,GRASS_HERB,OTHER_FUELS_REDUCTION,2022,100.0,NaN,100.0
170,NPS,SHRB_CHAP,OTHER_FUELS_REDUCTION,2022,13.0,NaN,13.0
183,NPS,SPARSE,OTHER_FUELS_REDUCTION,2022,30.0,NaN,30.0
204,NPS,URBAN,OTHER_FUELS_REDUCTION,2022,45.0,NaN,45.0


In [10]:
reference_gdf = gpd.read_file(validation_gdb_path, driver='openFileGDB', layer=reference_layer_name)
output_gdf = gpd.read_file(output_gdb_path, driver='openFileGDB', layer=output_layer_name)

C:\Users\sky\.conda\envs\its_recode\Lib\site-packages\pyogrio\raw.py:198: RuntimeWarning: driver OpenFileGDB does not support open option DRIVER
  return ogr_read(
C:\Users\sky\.conda\envs\its_recode\Lib\site-packages\pyogrio\raw.py:198: RuntimeWarning: driver OpenFileGDB does not support open option DRIVER
  return ogr_read(


In [12]:
col_sum='ACTIVITY_QUANTITY'
group=['ADMINISTERING_ORG', 'BROAD_VEGETATION_TYPE', 'PRIMARY_OBJECTIVE', "YEAR_TXT"]

In [3]:
output_gdb_path = r'D:\WORK\wildfire\Interagency-Tracking-System\its\ITSGDB_backup\V2.0\USFS_1950_2025.gdb'

In [6]:
gpd.list_layers(output_gdb_path)

,name,geometry_type
0,USFS_Region04_enriched_20250509,MultiPolygon
1,USFS_Region05_enriched_20250509,MultiPolygon
2,USFS_Region06_enriched_20250509,MultiPolygon


In [17]:
output_layer_name = 'USFS_Region05_enriched_20250509'

In [18]:
output_gdf = gpd.read_file(output_gdb_path, driver='openFileGDB', layer=output_layer_name)
output_gdf.columns = [col.upper() for col in output_gdf.columns]

C:\Users\sky\.conda\envs\its_recode\Lib\site-packages\pyogrio\raw.py:198: RuntimeWarning: driver OpenFileGDB does not support open option DRIVER
  return ogr_read(
C:\Users\sky\.conda\envs\its_recode\Lib\site-packages\pyogrio\raw.py:198: RuntimeWarning: organizePolygons() received a polygon with more than 100 parts. The processing may be really slow.  You can skip the processing by setting METHOD=SKIP, or only make it analyze counter-clock wise parts by setting METHOD=ONLY_CCW if you can assume that the outline of holes is counter-clock wise defined
  return ogr_read(


In [19]:
output_gdf[output_gdf['COUNTS_TO_MAS'] == 'YES'][[col_sum] + group].groupby(group).sum().reset_index()

,ADMINISTERING_ORG,BROAD_VEGETATION_TYPE,PRIMARY_OBJECTIVE,YEAR_TXT,ACTIVITY_QUANTITY
0,USFS,AGRICULTURE,OTHER_FOREST_MGMT,1996,23.0
1,USFS,AGRICULTURE,OTHER_FOREST_MGMT,1998,24.0
2,USFS,AGRICULTURE,OTHER_FOREST_MGMT,1999,24.0
3,USFS,AGRICULTURE,OTHER_FOREST_MGMT,2000,32.0
4,USFS,AGRICULTURE,OTHER_FOREST_MGMT,2007,42.0
...,...,...,...,...,...
1429,USFS,WETLAND,REFORESTATION,1996,12.0
1430,USFS,WETLAND,REFORESTATION,1998,5.0
1431,USFS,WETLAND,REFORESTATION,1999,12.0
1432,USFS,WETLAND,REFORESTATION,2001,3.0
